## Astrowidgets test

Craig Lage - 03Jan23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from lsst.daf.butler import Butler
import lsst.afw.display
import lsst.display.astrowidgets
lsst.afw.display.setDefaultBackend("astrowidgets")

from lsst.summit.utils import BestEffortIsr
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    #vmin = 1
    #vmax = 50
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

def plotExpAstrowidgets(exp, display):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    exp.image.array = data
    print(vmin, vmax)
    display.scale('linear', vmin, max=vmax)
    display.mtv(exp)
    return

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return


In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])
best_effort_isr = BestEffortIsr(embargo=True)

In [ ]:
expId = 2022121200752

 ### Old ISR

In [ ]:
dataId = {'exposure': expId, 'detector': 0, "instrument": 'LATISS'}
best_effort_isr.doWrite = False  # Don't write to butler database
exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
fig = plotExp(exp)
plt.suptitle(f"{expId}")

In [ ]:
display = lsst.afw.display.Display(dims=(815, 800))  # size in screen pixels
display.embed()

In [ ]:
expId = 2023020700170 # Using parallel overscan
exp = butler.get('raw', detector=0, exposure=expId)

plotExpAstrowidgets(exp, display)